In [1]:
# Import Dependencies
from __future__ import unicode_literals, print_function, division
import glob
from io import open
import numpy as np
import pandas as pd

In [2]:
# Load and Visualize some data files
# Arabic Names
df = pd.read_csv('./data/names/Arabic.txt', header=None)
df.head()

,0
0,Khoury
1,Nahas
2,Daher
3,Gerges
4,Nazari


In [3]:
# Load and Visualize some data files
# English Names
df = pd.read_csv('./data/names/English.txt', header=None)
df.head()

,0
0,Abbas
1,Abbey
2,Abbott
3,Abdi
4,Abel


In [4]:
# Load and Visualize some data files
# French Names
df = pd.read_csv('./data/names/French.txt', header=None)
df.head()

,0
0,Abel
1,Abraham
2,Adam
3,Albert
4,Allard


In [5]:
categorical_names = {}
for fileName in glob.glob('./data/names/*'):
    dictKey = fileName.split('/')[3].split('.')[0]
    txt = open(fileName, encoding='utf-8').read().strip().split('\n')
    categorical_names[dictKey] = txt

In [6]:
categorical_names['Czech'][:5]

['Abl', 'Adsit', 'Ajdrna', 'Alt', 'Antonowitsch']

In [7]:
categorical_names['Arabic'][:5]

['Khoury', 'Nahas', 'Daher', 'Gerges', 'Nazari']

In [8]:
categorical_names['English'][:5]

['Abbas', 'Abbey', 'Abbott', 'Abdi', 'Abel']